# Federated Learning Training Plan: Create Plan

Let's try to make protobuf-serializable Training Plan and Model that work after deserializing :)

Current list of problems:
 * No support for autograd in Plan tracing (.backward() doesn't work inside the Plan).
 * `tensor.shape` value seem to be recorded as constant during Plan tracing, so we need to pass `batch_size`, can't take it from tensor itself.
 * Plan needs a list of all Model params in the argument list, it would be nicer if this list is figured out automatically so you just pass the Model (not sure it's solvable jit might not accept the model as ScriptModule input?)
 * Plan doesn't return input args from Plan, e.g. if they were updated with inplace operation
 * others? 


In [1]:
%load_ext autoreload
%autoreload 2
import syft as sy
import torch as th
from torchvision import datasets, transforms
from torch import jit
from syft.serde import protobuf
import os
from syft.execution.state import State
from syft.execution.placeholder import PlaceHolder


sy.hook(globals())
# force protobuf serialization for tensors
hook.local_worker.framework = None
th.random.manual_seed(1)

C:\Users\Vova\AppData\Local\conda\conda\envs\pysyft\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Vova\AppData\Local\conda\conda\envs\pysyft\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Vova\AppData\Local\conda\conda\envs\pysyft\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Vova\AppData\Loc

Setting up Sandbox...
Done!


This utility function will serialize any object to protobuf binary and save to a file.

In [2]:
def serializeToBinPb(worker, obj, filename):
    pb = protobuf.serde._bufferize(worker, obj)
    bin = pb.SerializeToString()
    print("Writing %s to %s/%s" % (obj.__class__.__name__, os.getcwd(), filename))
    with open(filename, "wb") as f:
        f.write(bin)

## Step 1: Define the model

This model will train on MNIST data, it's very simple yet can demonstrate learning process.
There're 2 linear layers:

* Linear 784x392
* ReLU
* Linear 392x10

Not using nn.Module or nn.Linear for now, just vanilla class and tensors.
No autograd, gradients are hand-coded.

As no loops supported inside Plan,
we can't iterate over parameters so everything that works with params
(get/set, step) is moved into the model to make Plan code more generic.

In [3]:
class Net():
    def __init__(self):
        super(Net, self).__init__()
        self.W1 = th.randn(392, 784) / th.sqrt(th.tensor(784.))
        self.b1 = th.zeros(392)
        self.W2 = th.randn(10, 392) / th.sqrt(th.tensor(392.))
        self.b2 = th.zeros(10)
        self.update_fn = None

    def forward(self, X):
        self.Z1 = X @ self.W1.t() + self.b1
        self.A1 = th.nn.functional.relu(self.Z1)
        self.Z2 = self.A1 @ self.W2.t() + self.b2
        return self.Z2

    def get_params(self):
        return self.W1, self.b1, self.W2, self.b2

    def set_params(self, *model_params):
        self.W1, self.b1, self.W2, self.b2 = model_params

    def grad(self, X, error):
        Z1_grad = (error @ self.W2) * (self.Z1 > 0).float()
        W1_grad = Z1_grad.t() @ X
        b1_grad = Z1_grad.sum(0)
        W2_grad = error.t() @ self.A1
        b2_grad = error.sum(0)
        return W1_grad, b1_grad, W2_grad, b2_grad

model = Net()

## Step 2: Define Training Plan
### Loss function
Batch size needs to be passed because otherwise `target.shape[0]` will be saved as `1` constant during Plan trace with dummy data.
Grad is also returned here.

In [4]:
def cross_entropy_with_logits(output, target, batch_size):
    probs = th.nn.functional.softmax(output, dim=1)
    loss = -(target * th.log(probs)).mean()
    loss_grad = (probs - target) / (batch_size * target.shape[1])
    return probs, loss, loss_grad

### Optimization function

Just updates weights with grad*lr.

Note: can't do inplace update and return this value from the Plan, which is potentially bad for memory.

In [5]:
def naive_sgd(param, grad, **kwargs):
    return param - grad * kwargs['lr']

### Training Plan procedure

In [6]:
model_params = model.get_params()

# define plan input dimensions
X_size = (-1, 784)
y_size = (-1, 10)
scalar_size = (1,)
model_params_shapes = [p.shape for p in model_params]

args_shape = [
    X_size,  # X
    y_size,  # y
    scalar_size,  # batch_size
    scalar_size,  # lr
    *model_params_shapes  # *model_params
]

#@sy.func2plan(args_shape=args_shape)
def training_plan(X, y, batch_size, lr, *model_params):
    # inject params into model
    model.set_params(*model_params)

    # forward pass
    output = model.forward(X)

    # loss
    probs = th.nn.functional.softmax(output, dim=1)
    loss_op_log = th.log(probs)
    loss_op_mul = y * loss_op_log
    loss_op_sum = loss_op_mul.sum()
    loss_op_div = loss_op_sum / (batch_size * y.shape[1])
    loss = -loss_op_div
    loss_grad = (probs - y) / (batch_size * y.shape[1])

    # backprop
    grads = model.grad(X, loss_grad)

    # step
    updated_params = [naive_sgd(param, grads[i], lr=lr)
                      for i, param in enumerate(model_params)]

    # accuracy
    pred = th.argmax(probs, dim=1)
    target = th.argmax(y, dim=1)
    acc = pred.eq(target).float().sum() / batch_size

    return (
        loss,
        acc,
        *updated_params,
        probs, loss_op_log, loss_op_mul, loss_op_sum, loss_op_div
    )

Let's look inside the Plan and print out the list of operations recorded.

Plan should be built to contain both "list of ops" and torchscript versions.

In [7]:
batch_size = th.tensor(1)
lr = th.tensor(0.01)
X = th.randn(1, 784)
y = th.randn(1, 10)
torchscript_plan = th.jit.trace(training_plan, (X, y, batch_size, lr, *model_params))
print(torchscript_plan.code)

def training_plan(argument_0: Tensor,
    argument_1: Tensor,
    argument_2: Tensor,
    argument_3: Tensor,
    argument_4: Tensor,
    argument_5: Tensor,
    argument_6: Tensor,
    argument_7: Tensor) -> Tuple[Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor]:
  _0 = torch.matmul(argument_0, torch.t(argument_4))
  _1 = torch.add(_0, argument_5, alpha=1)
  _2 = torch.relu(_1)
  _3 = torch.add(torch.matmul(_2, torch.t(argument_6)), argument_7, alpha=1)
  _4 = torch.softmax(_3, 1, None)
  _5 = torch.log(_4)
  _6 = torch.mul(argument_1, _5)
  _7 = torch.sum(_6, dtype=None)
  _8 = ops.prim.NumToTensor(torch.size(argument_1, 1))
  _9 = torch.div(_7, torch.mul(argument_2, _8))
  _10 = torch.neg(_9)
  _11 = ops.prim.NumToTensor(torch.size(argument_1, 1))
  _12 = torch.div(torch.sub(_4, argument_1, alpha=1), torch.mul(argument_2, _11))
  _13 = torch.matmul(_12, argument_6)
  _14 = torch.to(torch.gt(_1, 0), 6, False, False, None)
  _15 = torch.mul(_13, 

In [8]:
mnist = th.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=1,
    shuffle=False
)

X, y = next(iter(mnist))
X = X.view(X.shape[0], -1)
y_oh = th.nn.functional.one_hot(y, 10)
loss, acc, *rest = torchscript_plan(X, y_oh, batch_size, lr, *model_params)
print('loss', loss)
print('acc', acc)

for i, name in enumerate(['probs', 'loss_op_log', 'loss_op_mul', 'loss_op_sum', 'loss_op_div']):
    print(name, rest[-5 + i].shape, rest[-5 + i])

loss tensor(0.2356)
acc tensor(0.)
probs torch.Size([1, 10]) tensor([[0.0973, 0.1180, 0.1267, 0.0966, 0.0886, 0.0948, 0.0901, 0.0840, 0.1043,
         0.0996]])
loss_op_log torch.Size([1, 10]) tensor([[-2.3300, -2.1370, -2.0655, -2.3374, -2.4235, -2.3562, -2.4064, -2.4773,
         -2.2604, -2.3070]])
loss_op_mul torch.Size([1, 10]) tensor([[-0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -2.3562, -0.0000, -0.0000,
         -0.0000, -0.0000]])
loss_op_sum torch.Size([]) tensor(-2.3562)
loss_op_div torch.Size([]) tensor(-0.2356)


In [9]:
# Save torchscript as plain file
torchscript_plan.save("ts_plan.pt")

## Step 3: Serialize!

Now it's time to serialize model params and plans to protobuf and save them for further usage:
 * In "Execute Plan" notebook, we load and execute these plans & model, from Python.
 * In "Host Plan" notebook, we send these plans & model to PyGrid, so it can be executed from other worker (e.g. syft.js).

**NOTE:**
 * We don't serialize full Model, only weights. How the Model is serialized is TBD.
   State is suitable protobuf class to wrap list of Model params tensors.

In [10]:
# serializeToBinPb(hook.local_worker, training_plan, "tp_full.pb")

# wrap weights in State to serialize
# model_params_state = State(
#     owner=hook.local_worker,
#     state_placeholders=[PlaceHolder().instantiate(param) for param in model_params]
# )
#
# serializeToBinPb(hook.local_worker, model_params_state, "model_params.pb")
